In [1]:
# read files and combine
import pandas as pd
import re
from IPython.core.display import display, HTML

In [2]:
#read files
files = ['glassdoor_Business_Analyst.csv', 'glassdoor_Data_Analyst.csv', 'glassdoor_Data_Architect.csv', 'glassdoor_Data_Engineer.csv', 'glassdoor_Data_Scientist.csv', 'glassdoor_Machine_Learning_Engineer.csv']
dfs = [pd.read_csv(file) for file in files]
data_df = pd.concat(dfs, ignore_index=True)
print(data_df.head())

                      Job Title                           Salary Estimate  \
0              Business Analyst              $61K - $85K (Glassdoor est.)   
1      Business Systems Analyst  $61.00 - $71.00 Per Hour (Employer est.)   
2     Business/Database Analyst              $58K - $92K (Glassdoor est.)   
3                  Data Analyst                      $60K (Employer est.)   
4  Junior Business/Data Analyst              $51K - $78K (Glassdoor est.)   

                  Company Name        Location  \
0         BCVS group Inc.5.0 ★       Plano, TX   
1                          SRP      Irvine, CA   
2     JSR Tech Consulting5.0 ★      Newark, NJ   
3                 Wildcat3.5 ★    New York, NY   
4  The Kenific Group, Inc2.6 ★  Washington, DC   

                                     Job Description  Rating  \
0  Hello,\r\nWe are hiring for Business System An...     5.0   
1                                                 -1     5.0   
2  JSR has an immediate opening for their di

In [3]:
data_df.shape

(1080, 12)

In [4]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          1080 non-null   object 
 1   Salary Estimate    1080 non-null   object 
 2   Company Name       1080 non-null   object 
 3   Location           1080 non-null   object 
 4   Job Description    1080 non-null   object 
 5   Rating             1080 non-null   float64
 6   Size               1080 non-null   object 
 7   Founded            1080 non-null   int64  
 8   Type of ownership  1080 non-null   object 
 9   Industry           1080 non-null   object 
 10  Sector             1080 non-null   object 
 11  Revenue            1080 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 101.4+ KB


In [5]:
# clean duplicate and salary
data_clean = data_df.drop_duplicates()
data_clean = data_clean[data_clean['Salary Estimate'] != "-1"]
data_clean.shape

(563, 12)

In [6]:
data_clean['hourly'] = data_clean['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
data_clean['employer_est'] = data_clean['Salary Estimate'].apply(lambda x: 1 if 'employer est.' in x.lower() else 0)
data_clean['glassdoor_est'] = data_clean['Salary Estimate'].apply(lambda x: 1 if 'glassdoor est.' in x.lower() else 0)

In [7]:
def extract_min_max_salary(salary_string):
    # Remove the 'K' (thousand) sign and '$' sign
    salary_string = salary_string.replace('K', '').replace('$', '')

    # Extract the numbers (minimum and maximum salary)
    salary_range = re.findall("\d+\.\d+|\d+", salary_string)

    # If there is no range (only one number), then the minimum and maximum salary are the same
    if len(salary_range) == 1:
        min_salary = max_salary = float(salary_range[0])

    # If there is a range (two numbers), the first is the minimum salary and the second is the maximum salary
    elif len(salary_range) == 2:
        min_salary, max_salary = map(float, salary_range)

    else:
        min_salary = max_salary = None

    return min_salary, max_salary


In [8]:
data_clean['min_salary'], data_clean['max_salary'] = zip(*data_clean['Salary Estimate'].map(extract_min_max_salary))
data_clean.head()

,Job Title,Salary Estimate,Company Name,Location,Job Description,Rating,Size,Founded,Type of ownership,Industry,Sector,Revenue,hourly,employer_est,glassdoor_est,min_salary,max_salary
0,Business Analyst,$61K - $85K (Glassdoor est.),BCVS group Inc.5.0 ★,"Plano, TX","Hello,\r\nWe are hiring for Business System An...",5.0,Unknown,-1,Company - Public,-1,-1,Unknown / Non-Applicable,0,0,1,61.0,85.0
1,Business Systems Analyst,$61.00 - $71.00 Per Hour (Employer est.),SRP,"Irvine, CA",-1,5.0,-1,-1,-1,-1,-1,-1,1,1,0,61.0,71.0
2,Business/Database Analyst,$58K - $92K (Glassdoor est.),JSR Tech Consulting5.0 ★,"Newark, NJ",JSR has an immediate opening for their direct ...,5.0,51 to 200 Employees,2015,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD),0,0,1,58.0,92.0
3,Data Analyst,$60K (Employer est.),Wildcat3.5 ★,"New York, NY",Are you someone who loves crunching numbers an...,3.5,201 to 500 Employees,1972,Nonprofit Organization,Education & Training Services,Education,$5 to $25 million (USD),0,1,0,60.0,60.0
4,Junior Business/Data Analyst,$51K - $78K (Glassdoor est.),"The Kenific Group, Inc2.6 ★","Washington, DC","Company Overview:\r\nThe Kenific Group, Inc. (...",2.6,51 to 200 Employees,-1,Company - Private,Business Consulting,Management & Consulting,Less than $1 million (USD),0,0,1,51.0,78.0


In [9]:
#hourly to annually
data_clean.loc[data_clean['hourly'] == 1, 'min_salary'] = data_clean['min_salary'] * 2
data_clean.loc[data_clean['hourly'] == 1, 'max_salary'] = data_clean['max_salary'] * 2
data_clean.head()

,Job Title,Salary Estimate,Company Name,Location,Job Description,Rating,Size,Founded,Type of ownership,Industry,Sector,Revenue,hourly,employer_est,glassdoor_est,min_salary,max_salary
0,Business Analyst,$61K - $85K (Glassdoor est.),BCVS group Inc.5.0 ★,"Plano, TX","Hello,\r\nWe are hiring for Business System An...",5.0,Unknown,-1,Company - Public,-1,-1,Unknown / Non-Applicable,0,0,1,61.0,85.0
1,Business Systems Analyst,$61.00 - $71.00 Per Hour (Employer est.),SRP,"Irvine, CA",-1,5.0,-1,-1,-1,-1,-1,-1,1,1,0,122.0,142.0
2,Business/Database Analyst,$58K - $92K (Glassdoor est.),JSR Tech Consulting5.0 ★,"Newark, NJ",JSR has an immediate opening for their direct ...,5.0,51 to 200 Employees,2015,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD),0,0,1,58.0,92.0
3,Data Analyst,$60K (Employer est.),Wildcat3.5 ★,"New York, NY",Are you someone who loves crunching numbers an...,3.5,201 to 500 Employees,1972,Nonprofit Organization,Education & Training Services,Education,$5 to $25 million (USD),0,1,0,60.0,60.0
4,Junior Business/Data Analyst,$51K - $78K (Glassdoor est.),"The Kenific Group, Inc2.6 ★","Washington, DC","Company Overview:\r\nThe Kenific Group, Inc. (...",2.6,51 to 200 Employees,-1,Company - Private,Business Consulting,Management & Consulting,Less than $1 million (USD),0,0,1,51.0,78.0


In [10]:
data_clean[data_clean['hourly'] == 1][['min_salary', 'max_salary']]

,min_salary,max_salary
1,122.00,142.00
7,124.00,124.00
16,46.00,50.00
20,60.00,60.00
21,100.00,110.00
...,...,...
649,140.00,160.00
724,91.92,137.90
778,52.40,102.78
796,52.40,102.78


In [11]:
data_clean['avg_salary'] = (data_clean.min_salary+data_clean.max_salary)/2

In [12]:
#age of company 
data_clean['age'] = data_clean.Founded.apply(lambda x: x if x <1 else 2023 - x)

In [13]:
# Define a function to clean the company name
def clean_company_name(name):
    # Check if the last 5 characters match the pattern of a rating (e.g., "5.0 ★")
    if name[-5:-2].replace('.', '').isdigit() and name[-1] == "★":
        return name[:-5].strip()
    else:
        return name.strip()

In [14]:
#Company name text only
data_clean['company_txt'] = data_clean["Company Name"].apply(clean_company_name)

In [15]:
data_clean.head()

,Job Title,Salary Estimate,Company Name,Location,Job Description,Rating,Size,Founded,Type of ownership,Industry,Sector,Revenue,hourly,employer_est,glassdoor_est,min_salary,max_salary,avg_salary,age,company_txt
0,Business Analyst,$61K - $85K (Glassdoor est.),BCVS group Inc.5.0 ★,"Plano, TX","Hello,\r\nWe are hiring for Business System An...",5.0,Unknown,-1,Company - Public,-1,-1,Unknown / Non-Applicable,0,0,1,61.0,85.0,73.0,-1,BCVS group Inc.
1,Business Systems Analyst,$61.00 - $71.00 Per Hour (Employer est.),SRP,"Irvine, CA",-1,5.0,-1,-1,-1,-1,-1,-1,1,1,0,122.0,142.0,132.0,-1,SRP
2,Business/Database Analyst,$58K - $92K (Glassdoor est.),JSR Tech Consulting5.0 ★,"Newark, NJ",JSR has an immediate opening for their direct ...,5.0,51 to 200 Employees,2015,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD),0,0,1,58.0,92.0,75.0,8,JSR Tech Consulting
3,Data Analyst,$60K (Employer est.),Wildcat3.5 ★,"New York, NY",Are you someone who loves crunching numbers an...,3.5,201 to 500 Employees,1972,Nonprofit Organization,Education & Training Services,Education,$5 to $25 million (USD),0,1,0,60.0,60.0,60.0,51,Wildcat
4,Junior Business/Data Analyst,$51K - $78K (Glassdoor est.),"The Kenific Group, Inc2.6 ★","Washington, DC","Company Overview:\r\nThe Kenific Group, Inc. (...",2.6,51 to 200 Employees,-1,Company - Private,Business Consulting,Management & Consulting,Less than $1 million (USD),0,0,1,51.0,78.0,64.5,-1,"The Kenific Group, Inc"


In [16]:
data_clean['State'] = data_clean['Location'].str.split(',').apply(lambda x: x[1].strip() if len(x) > 1 else x[0])

In [17]:
data_clean.State.value_counts()

CA               105
Remote            89
NY                52
TX                51
NJ                21
WA                20
FL                19
VA                16
MI                15
NC                14
MA                14
MD                13
GA                13
DC                12
AZ                11
OH                11
MO                10
PA                 8
MN                 8
KY                 7
United States      6
IL                 5
AR                 4
CO                 4
KS                 4
AL                 4
IN                 2
NE                 2
California         2
IA                 2
UT                 2
LA                 2
OR                 2
AK                 2
WI                 2
TN                 1
DE                 1
Missouri           1
Pennsylvania       1
SC                 1
ND                 1
ME                 1
HI                 1
NM                 1
Name: State, dtype: int64

In [18]:
#change state name
data_clean['State'] = data_clean['State'].replace('Missouri', 'MO')
data_clean['State'] = data_clean['State'].replace('California', 'CA')
data_clean['State'] = data_clean['State'].replace('Pennsylvania', 'PA')
data_clean['State'] = data_clean['State'].replace('United States', 'Remote')

In [19]:
state_counts=data_clean.State.value_counts().to_frame().transpose()
# Display horizontally with scroll
display(HTML('<div style="max-width: 800px; overflow-x: auto; border: 1px solid #ccc;">' +
             state_counts.to_html(index=False) +
             '</div>'))


CA,Remote,NY,TX,NJ,WA,FL,VA,MI,NC,MA,MD,GA,DC,AZ,MO,OH,PA,MN,KY,IL,CO,KS,AR,AL,UT,AK,NE,IA,WI,LA,IN,OR,TN,SC,ME,ND,HI,NM,DE
107,95,52,51,21,20,19,16,15,14,14,13,13,12,11,11,11,9,8,7,5,4,4,4,4,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1


In [20]:
#Job cleaning
data_clean['Job Title'].value_counts()

Business Analyst                          45
Data Engineer                             45
Data Scientist                            32
Data Analyst                              29
Senior Data Engineer                      16
                                          ..
Software Architect (REMOTE)                1
AWS Data Engineer                          1
Sr Data Platform Architect - Remote AZ     1
Data Architect (Azure)                     1
Staff Data Scientist                       1
Name: Job Title, Length: 265, dtype: int64

In [21]:
def title_simplifier(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'data analyst' in title.lower():
        return 'data analyst'
    elif 'business analyst' in title.lower():
        return 'business analyst'
    elif 'architect' in title.lower():
        return 'architect'
    elif 'machine learning' in title.lower():
        return 'mle'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else:
        return 'na'

def employeetype(title):
    if pd.isna(title):
        return "Unknown"  # Handle NaN titles

    title = title.lower()
    emp = ['sr', 'senior', 'staff', 'lead', 'principal', 'vp', 'vice president', 'director']
    
    for i in emp:
        if i in title:
            return "senior_emp"
    
    return "junior_emp"

In [22]:
data_clean['job_simp'] = data_clean['Job Title'].apply(title_simplifier)

data_clean.job_simp.value_counts()

na                  124
data engineer       122
business analyst     76
data scientist       67
data analyst         66
architect            60
mle                  47
director              1
Name: job_simp, dtype: int64

In [23]:
data_clean['employee_expriece']= data_clean['Job Title'].apply(employeetype)
data_clean.employee_expriece.value_counts()

junior_emp    457
senior_emp    106
Name: employee_expriece, dtype: int64

In [24]:
#Create lenth of JD
data_clean['desc_len'] = data_clean['Job Description'].apply(lambda x: len(x))

In [25]:
#Extract skill from JD
data_clean['python']=data_clean['Job Description'].apply(lambda x: 1 if "python" in x.lower() else 0)
data_clean['excel']=data_clean['Job Description'].apply(lambda x: 1 if "excel" in x.lower() else 0)
data_clean['sql']=data_clean['Job Description'].apply(lambda x: 1 if "sql" in x.lower() else 0)
data_clean['tableau']=data_clean['Job Description'].apply(lambda x: 1 if "tableau" in x.lower() else 0)
data_clean['spark']=data_clean['Job Description'].apply(lambda x: 1 if "spark" in x.lower() else 0)
data_clean['machine learning']=data_clean['Job Description'].apply(lambda x: 1 if "machine learning" in x.lower() else 0)
data_clean['aws']=data_clean['Job Description'].apply(lambda x: 1 if "aws" in x.lower() else 0)

In [26]:
data_clean = data_clean.drop(columns=['Salary Estimate', 'Company Name'])

In [27]:
data_clean

,Job Title,Location,Job Description,Rating,Size,Founded,Type of ownership,Industry,Sector,Revenue,...,job_simp,employee_expriece,desc_len,python,excel,sql,tableau,spark,machine learning,aws
0,Business Analyst,"Plano, TX","Hello,\r\nWe are hiring for Business System An...",5.0,Unknown,-1,Company - Public,-1,-1,Unknown / Non-Applicable,...,business analyst,junior_emp,1459,0,1,1,0,0,0,0
1,Business Systems Analyst,"Irvine, CA",-1,5.0,-1,-1,-1,-1,-1,-1,...,na,junior_emp,2,0,0,0,0,0,0,0
2,Business/Database Analyst,"Newark, NJ",JSR has an immediate opening for their direct ...,5.0,51 to 200 Employees,2015,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD),...,na,junior_emp,4188,0,1,1,0,0,0,1
3,Data Analyst,"New York, NY",Are you someone who loves crunching numbers an...,3.5,201 to 500 Employees,1972,Nonprofit Organization,Education & Training Services,Education,$5 to $25 million (USD),...,data analyst,junior_emp,3518,0,1,0,0,0,0,0
4,Junior Business/Data Analyst,"Washington, DC","Company Overview:\r\nThe Kenific Group, Inc. (...",2.6,51 to 200 Employees,-1,Company - Private,Business Consulting,Management & Consulting,Less than $1 million (USD),...,data analyst,junior_emp,3353,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,"Software Engineer, Machine Learning Operations","Redmond, WA","Email, chat, and the web are an integral parts...",4.3,10000+ Employees,1975,Company - Public,Computer Hardware Development,Information Technology,$10+ billion (USD),...,mle,junior_emp,5149,1,0,1,0,0,1,1
1070,Computer Vision Engineer,"Fort Belvoir, VA",Company Overview:\r\nWe are a world-class team...,3.7,501 to 1000 Employees,1956,Subsidiary or Business Segment,Aerospace & Defense,Aerospace & Defense,$100 to $500 million (USD),...,na,junior_emp,4122,1,1,0,0,1,1,0
1073,"Engineering Manager, Machine Learning Platform",Remote,"At Coinbase, our mission is to increase econom...",3.7,501 to 1000 Employees,2012,Company - Public,Internet & Web Services,Information Technology,Unknown / Non-Applicable,...,mle,junior_emp,6541,0,0,0,0,1,1,0
1077,Applied Scientist II,"Redmond, WA",Do you have a passion for data and machine lea...,4.3,10000+ Employees,1975,Company - Public,Computer Hardware Development,Information Technology,$10+ billion (USD),...,na,junior_emp,4831,1,0,1,0,0,1,1


In [28]:
data_clean.to_csv('data_cleaned.csv', index = False)